##1. Installing Libraries and extracting Dataset

In [ ]:
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI


In [ ]:
!pip install pycocotools

In [ ]:
!pip install --upgrade scikit-image scipy

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
import seaborn as sns
import numpy as np

from random import shuffle
from PIL import Image

from pycocotools.coco import COCO

##2. Extracting Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Load COCO annotation file (you can also use the test file)
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d /content/drive/MyDrive/COCO_dataset/annotations

# Initialize COCO API for instance annotations
annFile = '/content/drive/MyDrive/COCO_dataset/annotations/annotations/instances_train2017.json'
coco = COCO(annFile)

I have added a cell to mount your Google Drive. Please run it and authorize Colab to access your Drive.

Now, I will modify the code in cell `E7SYS1ZFrFoo` to extract the annotations to `/content/drive/MyDrive/COCO_dataset/annotations` and update the `annFile` variable to reflect this path.

Here is the code to load COCO annotations and display the categories.

**Important:** The `annotation_path` variable needs to be set to the actual path of your COCO annotation file.

In [ ]:
import pandas as pd

In [ ]:
ANNOT_URL = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
ANNOT_DIR = "./annotations"
ANNOT_FILE = f"{ANNOT_DIR}/annotations/instances_train2017.json"

if not os.path.exists(ANNOT_FILE):
    !wget -q $ANNOT_URL -O annotations.zip
    !unzip -q annotations.zip -d $ANNOT_DIR

# --- Step 4: Initialize COCO API ---
coco = COCO(ANNOT_FILE)

# --- Step 5: Get all categories ---
cat_ids = coco.getCatIds()
cats = coco.loadCats(cat_ids)

# --- Step 6: For each category, count how many images belong to it ---
category_counts = []
for cat in cats:
    img_ids = coco.getImgIds(catIds=[cat['id']])
    category_counts.append({
        'Category': cat['name'],
        'Supercategory': cat['supercategory'],
        'Num_Images': len(img_ids)
    })

# --- Step 7: Display in tabular format ---
df = pd.DataFrame(category_counts).sort_values(by='Num_Images', ascending=False).reset_index(drop=True)

# Set Pandas display options to ensure tabular view
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# --- Step 8: Display nicely ---
from IPython.display import display
display(df.style.set_properties(**{'text-align': 'center'}))

# Optional: quick summary
print(f"\nTotal Categories: {len(df)}")
unique_imgs = len(set(sum([coco.getImgIds(catIds=[c['id']]) for c in cats], [])))
print(f"Total Unique Images (approx, may overlap): {unique_imgs}")

In [ ]:
from pycocotools.coco import COCO
import pandas as pd
import requests

# Load captions annotation file
!wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d ./annotations

annFile = './annotations/annotations/captions_train2017.json'
coco_caps = COCO(annFile)

# Get image-caption pairs (limit to 2000 for now)
anns = coco_caps.loadAnns(coco_caps.getAnnIds())[:2000]
data = []
for a in anns:
    img = coco_caps.loadImgs(a['image_id'])[0]
    data.append({
        "image_url": img['coco_url'],
        "caption": a['caption']
    })

df = pd.DataFrame(data)
df.head()


##3. Model Pipeline

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# import torch
# import clip
# from PIL import Image

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)

# text_inputs = torch.cat([clip.tokenize(df['caption'].iloc[i]) for i in range(5)]).to(device)
# with torch.no_grad():
#     text_features = model.encode_text(text_inputs)
# print("Embedding shape:", text_features.shape)


In [ ]:
import pandas as pd
from pycocotools.coco import COCO

# Ensure annotations directory exists
ANNOT_DIR = './annotations'
if not os.path.exists(ANNOT_DIR):
    os.makedirs(ANNOT_DIR)

# Download and unzip the annotations file if not already present
ANNOT_ZIP = 'annotations_trainval2017.zip'
if not os.path.exists(ANNOT_ZIP):
    !wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -o -q annotations_trainval2017.zip -d {ANNOT_DIR} # -o flag to overwrite without prompting

# Define the path to the captions annotation file
captions_ann_file = f'{ANNOT_DIR}/annotations/captions_train2017.json'

# Initialize COCO API for captions
coco_caps = COCO(captions_ann_file)

# Get image-caption pairs (limit to 2000 for now, as in the original code)
anns = coco_caps.loadAnns(coco_caps.getAnnIds())[:2000]

data = []
for a in anns:
    img = coco_caps.loadImgs(a['image_id'])[0]
    data.append({
        "image_url": img['coco_url'],
        "caption": a['caption']
    })

caption_df = pd.DataFrame(data)

print("Caption DataFrame created successfully with columns:", caption_df.columns.tolist())
display(caption_df.head())

In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Use caption_df that was created in the previous cell
text_inputs = torch.cat([clip.tokenize(caption_df['caption'].iloc[i]) for i in range(5)]).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
print("Embedding shape:", text_features.shape)

In [ ]:
# --- STEP 1: Install Dependencies ---
!pip install -q diffusers==0.31.0 transformers accelerate torch torchvision safetensors

# --- STEP 2: Imports ---
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import matplotlib.pyplot as plt

# --- STEP 3: Load Pretrained Stable Diffusion Model ---
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# --- STEP 4: Function to Generate Room Image ---
def generate_room(room_type, size, objects_list, style="modern"):
    prompt = f"A {style} {room_type} of size {size}, decorated with {', '.join(objects_list)}, realistic lighting, 4K interior render"
    print(f"\nPrompt → {prompt}")
    image = pipe(prompt, num_inference_steps=30, guidance_scale=8.5).images[0]
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    return image

# --- STEP 5: Example Run ---
generate_room(
    room_type="kitchen",
    size="10x12 ft",
    objects_list=["dining table", "refrigerator", "microwave", "plant"],
    style="cozy modern"
)


In [ ]:
# --- Step 1: Install dependencies (run once per session) ---
!pip install -q diffusers transformers accelerate safetensors torch torchvision

# --- Step 2: Import and load model ---
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# --- Step 3: Custom user prompt ---
prompt = input("Enter your prompt (e.g., 'A cozy 12x10 kitchen with fridge and plants'): ")

# --- Step 4: Generate and display ---
image = pipe(prompt, guidance_scale=7.5, num_inference_steps=30).images[0]
image.show()


In [ ]:
base_prompt = input("Enter base idea (e.g., 'modern kitchen'): ")
categories = ["refrigerator", "dining table", "lamp"]
prompt = f"A {base_prompt} with {', '.join(categories)}, 4K photo"
print("Final prompt:", prompt)

image = pipe(prompt, guidance_scale=7.5, num_inference_steps=30).images[0]
from IPython.display import display
display(image)


In [ ]:
import gradio as gr

def generate(prompt):
    image = pipe(prompt, guidance_scale=7.5, num_inference_steps=30).images[0]
    return image

gr.Interface(
    fn=generate,
    inputs=gr.Textbox(label="Describe your room"),
    outputs="image",
    title="Room Decor Generator"
).launch()


#2. Milestone 2

##4. Integrate text encoder (CLIP/BERT) with diffusion model (Stable Diffusion / DDPM)

In [ ]:
!pip install -q torch torchvision diffusers transformers accelerate safetensors numpy

import torch, numpy as np, os, json
from diffusers import StableDiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"


1. Loading pipeline with CLIP Encoder

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
).to(device)

tokenizer = pipe.tokenizer
text_encoder = pipe.text_encoder
print("Tokenizer & CLIP encoder loaded")

2. Generator or Encoding Function

In [ ]:
@torch.no_grad()
def encode_prompts(
    prompts,
    neg_prompts=None,
    device=device,
    max_length=77
):
    if neg_prompts is None:
        neg_prompts = [""] * len(prompts)

    # Tokenize into (batch, seq_len=77)
    text_inputs = tokenizer(
        prompts,
        padding="max_length",
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
    )

    neg_inputs = tokenizer(
        neg_prompts,
        padding="max_length",
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
    )

    # CLIP text encoder → (batch, 77, 768)
    prompt_embeds = text_encoder(text_inputs.input_ids.to(device))[0]
    neg_embeds    = text_encoder(neg_inputs.input_ids.to(device))[0]

    # Match UNet dtype
    dtype = pipe.unet.dtype
    prompt_embeds = prompt_embeds.to(dtype)
    neg_embeds    = neg_embeds.to(dtype)

    return prompt_embeds, neg_embeds

3. generating images using vecotor storage for embeddings

In [ ]:
@torch.no_grad()
def generate_images_from_embeds(
    prompt_embeds,
    negative_prompt_embeds,
    guidance_scale=7.5,
    num_inference_steps=30,
    height=512,
    width=512,
    seed=42,
):

    generator = torch.Generator(device=device).manual_seed(seed)

    out = pipe(
        prompt_embeds=prompt_embeds,
        negative_prompt_embeds=negative_prompt_embeds,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator,
        height=height,
        width=width,
        output_type="pil",
    )

    return out.images


4. Vector Storage

In [ ]:
def save_vector_store(prompts, prompt_embeds,
                      path_npz="data/vectorstore.npz",
                      path_json="data/vectorstore_meta.json"):
    os.makedirs("data", exist_ok=True)
    np.savez_compressed(path_npz, embeddings=prompt_embeds.cpu().numpy())
    with open(path_json, "w") as f:
        json.dump({"prompts": prompts}, f, indent=2)
    print("Vectorstore saved ✔")

def load_vector_store(path_npz="data/vectorstore.npz",
                      path_json="data/vectorstore_meta.json"):
    meta = json.load(open(path_json))
    arr = np.load(path_npz)["embeddings"]
    return meta["prompts"], arr

5. Testing CLIP with diffusion model

In [ ]:

prompts = [
    "A simple kitchen with refrigerator and dining table and lamp",
    "A modern living room with a smart TV and white sofa",
    "A bedroom with two lamps and a study desk",
    "A dining space with chandelier and wooden chairs",
    "A minimalist studio apartment with indoor plants"
]

neg = ["blurry, low quality, distorted"] * len(prompts)

# Encode embeddings
prompt_embeds, neg_embeds = encode_prompts(prompts, neg)

# Check shapes (SHOULD BE: (5, 77, 768))
print("Prompt embed shape:", prompt_embeds.shape)
print("Negative embed shape:", neg_embeds.shape)

# Save vectorstore
save_vector_store(prompts, prompt_embeds)

# Generate images
os.makedirs("milestone2_outputs", exist_ok=True)

images = generate_images_from_embeds(prompt_embeds, neg_embeds)

for i, img in enumerate(images):
    img.save(f"milestone2_outputs/sample_{i+1}.png")
    print("Saved image:", f"milestone2_outputs/sample_{i+1}.png")

##5. Run baseline conditional generation

In [ ]:
# ============================
# BASELINE CONDITIONAL GENERATION
# (Using pretrained Stable Diffusion)
# ============================

!pip install -q diffusers transformers accelerate torch safetensors

import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pretrained Stable Diffusion v1.5
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
).to(device)

# Your 5 milestone prompts
prompts = [
    "A simple kitchen with refrigerator and dining table and lamp",
    "A modern living room with a smart TV and white sofa",
    "A bedroom with two lamps and a study desk",
    "A dining space with chandelier and wooden chairs",
    "A minimalist studio apartment with indoor plants"
]

# Make output folder
os.makedirs("baseline_outputs", exist_ok=True)

# Generate baseline images
for i, prompt in enumerate(prompts):
    print(f"Generating image {i+1}...")
    image = pipe(prompt).images[0]
    image_path = f"baseline_outputs/baseline_{i+1}.png"
    image.save(image_path)
    print("Saved:", image_path)

print(" Baseline conditional generation complete!")


##6. Tune classifier-free guidance + noise schedule

In [ ]:
!pip install -q diffusers transformers accelerate torch safetensors

import torch
from diffusers import StableDiffusionPipeline
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16 if device=='cuda' else torch.float32
).to(device)

os.makedirs("cfg_scheduler_outputs", exist_ok=True)


2. Tuning Classifer free guidance

In [ ]:
prompts = [
    "A simple kitchen with refrigerator and dining table and lamp"
]

cfg_values = [3, 5, 7.5, 10, 12]

for cfg in cfg_values:
    print(f"Generating CFG={cfg}...")
    image = pipe(
        prompts[0],
        guidance_scale=cfg,
        num_inference_steps=30
    ).images[0]

    filename = f"cfg_scheduler_outputs/cfg_{cfg}.png"
    image.save(filename)
    print("Saved:", filename)


3. Tuning Diffusion free noise scheduler

In [ ]:
from diffusers import (
    DDIMScheduler,
    EulerAncestralDiscreteScheduler,
    LMSDiscreteScheduler,
    PNDMScheduler
)


In [ ]:
schedulers = {
    "PNDM": PNDMScheduler.from_config(pipe.scheduler.config),
    "DDIM": DDIMScheduler.from_config(pipe.scheduler.config),
    "EulerA": EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config),
    "LMS": LMSDiscreteScheduler.from_config(pipe.scheduler.config)
}

for name, sched in schedulers.items():
    print(f"Testing scheduler: {name}")
    pipe.scheduler = sched

    image = pipe(
        prompts[0],
        guidance_scale=7.5,  # keep constant
        num_inference_steps=30
    ).images[0]

    filename = f"cfg_scheduler_outputs/scheduler_{name}.png"
    image.save(filename)
    print("Saved:", filename)


#4. Git Repository Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!mkdir -p /content/drive/MyDrive/generative-ai-room-decorator/{data/raw,notebooks,src,outputs,configs}

In [ ]:
!touch generative-ai-room-decorator/{README.md,.gitignore}

In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!git init
!git remote add origin git@github.com:<username>/generative-ai-room-decorator.git


In [ ]:
!ssh-keygen -t ed25519 -C "pillalamarri.v@northeastern.edu"

# 2️⃣ Start SSH agent and add key
!eval "$(ssh-agent -s)"
!ssh-add ~/.ssh/id_ed25519

# 3️⃣ Show your public key
!cat ~/.ssh/id_ed25519.pub

In [ ]:
!eval "$(ssh-agent -s)"    # start SSH agent
!ssh-add ~/.ssh/id_ed25519 # add your SSH key to the agent


In [ ]:
!ssh -T git@github.com


In [ ]:
# === START: SSH Agent and Key Setup ===

# Start SSH agent and add key in one shell session
!eval "$(ssh-agent -s)"
!ssh-add ~/.ssh/id_ed25519

# Add GitHub to known_hosts (avoids "host key verification failed")
!mkdir -p ~/.ssh
!ssh-keyscan github.com >> ~/.ssh/known_hosts

# Set correct permissions
!chmod 700 ~/.ssh
!chmod 644 ~/.ssh/known_hosts

# Test SSH connection
!ssh -T git@github.com
# === END ===


In [ ]:
!git remote -v


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator


In [ ]:
!git rebase --abort


In [ ]:
!git status
!git branch


In [ ]:
!git switch -C main


In [ ]:
!git remote remove origin
!git remote add origin git@github.com:PVRPratyusha/generative-ai-room-decorator.git
!git fetch origin


In [ ]:
!git add .
!git commit -m "Clean branch reset after rebase issue"
!git push origin main --force


In [ ]:
!echo ".ipynb_checkpoints/" >> .gitignore
!git add .gitignore
!git commit -m "Ignore Colab checkpoint files"
!git push origin main


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator/notebooks
!cp Generative_Project_Milestone_1.ipynb Generative_Project_Milestone_1_broken_backup.ipynb


In [ ]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
!mv "/content/generative_project_milestone_1.py" "/content/drive/MyDrive/generative-ai-room-decorator/notebooks/"


In [ ]:
!ls /content


In [ ]:
!ls "/content/drive/MyDrive/generative-ai-room-decorator/notebooks"


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!git add notebooks/generative_project_milestone_1.py
!git commit -m "Re-upload working Milestone 1 notebook"
!git push origin main


In [ ]:
!touch data/.gitkeep
!touch src/.gitkeep
!touch outputs/.gitkeep
!touch configs/.gitkeep

%cd /content/drive/MyDrive/generative-ai-room-decorator
!git add .
!git commit -m "Add placeholder files to show folder structure"
!git push origin main


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator


In [ ]:
!git rm notebooks/Generative_Project_Milestone_1\ \(1\).ipynb \
        notebooks/Generative_Project_Milestone_1\ \(2\).ipynb \
        notebooks/Generative_Project_Milestone_1.ipynb \
        notebooks/Generative_Project_Milestone_1_broken_backup.ipynb


In [ ]:
!git commit -m "Remove redundant and corrupted notebook files"
!git push origin main


#Git Push for Milestone 2

In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator


In [ ]:
!git remote -v


In [ ]:
!git config --global user.email "pillalamarri.v@northeastern.edu"
!git config --global user.name "PVRPratyusha"


In [ ]:
!git checkout -b milestone2


In [ ]:
!git remote add origin https://<REMOVED>@github.com/PVRPratyusha/generative-ai-room-decorator.git

In [ ]:
!git remote remove origin


In [ ]:
# ============================
# 0️⃣  GO TO YOUR PROJECT FOLDER
# ============================
%cd /content/drive/MyDrive/generative-ai-room-decorator

# ============================
# 1️⃣  RE-INITIALIZE GIT SAFELY
# ============================
!rm -rf .git
!git init

# Set your identity
!git config --global user.email "pillalamarri.v@northeastern.edu"
!git config --global user.name "PVRPratyusha"

# ============================
# 2️⃣  ADD REMOTE (HTTPS + TOKEN)
# ============================
# 🔴 Replace YOUR_GITHUB_TOKEN_HERE
git remote add origin <REMOVED FOR SECURITY>


# Confirm remote
!git remote -v

# ============================
# 3️⃣  CREATE NEW BRANCH ONLY FOR MILESTONE 2
# ============================
!git checkout -b milestone2-colab

# ============================
# 4️⃣  ADD SEPARATE README FOR THIS BRANCH
# ============================
!echo "# Milestone 2 Work (Colab Branch)" > README_milestone2.md

# ============================
# 5️⃣  ADD ALL PROJECT FILES
# ============================
!git add .
!git commit -m "Milestone 2: Add full project structure + separate README for branch"

# ============================
# 6️⃣  PUSH ONLY THIS BRANCH
# ============================
!git push -u origin milestone2-colab


In [ ]:
!ls /content


In [ ]:
!pwd

In [ ]:
!find /content -name "*Milestone_2*.ipynb"


In [ ]:
!find /content/drive -name "*Milestone_2*.ipynb"


In [ ]:
!find / -name "*Milestone_2*.ipynb" 2>/dev/null


In [ ]:
!mv "/content/drive/MyDrive/Colab Notebooks/Generative_Project_Milestone_2.ipynb" "/content/drive/MyDrive/generative-ai-room-decorator/notebooks/"


In [ ]:
!ls "/content/drive/MyDrive/generative-ai-room-decorator/notebooks/"


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!git reset --soft HEAD~1


In [ ]:
!git add notebooks/Generative_Project_Milestone_2.ipynb


In [ ]:
!git commit -m "Add Milestone 2 notebook (token removed)"


In [ ]:
!git push origin milestone2-colab


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!git checkout milestone2-colab
!git reset --hard main


/content/drive/MyDrive/generative-ai-room-decorator
M	notebooks/Generative_Project_Milestone_2.ipynb
Already on 'milestone2-colab'
Your branch is ahead of 'origin/milestone2-colab' by 2 commits.
  (use "git push" to publish your local commits)
fatal: ambiguous argument 'main': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'


In [ ]:
!git checkout milestone2-colab
!git rm -r --cached .
!git clean -fd


M	notebooks/Generative_Project_Milestone_2.ipynb
Already on 'milestone2-colab'
Your branch is ahead of 'origin/milestone2-colab' by 2 commits.
  (use "git push" to publish your local commits)
rm '.gitignore'
rm 'README_milestone2.md'
rm 'configs/.gitkeep'
rm 'data/.gitkeep'
rm 'notebooks/Generative_Project_Milestone_2.ipynb'
rm 'notebooks/generative_project_milestone_1.py'
rm 'outputs/.gitkeep'
rm 'src/.gitkeep'
Removing .gitignore
Removing README_milestone2.md
Removing configs/
Removing data/
Removing notebooks/
Removing outputs/
Removing src/


In [ ]:
!git config --global user.email "pillalamarri.v@northeastern.edu"
!git config --global user.name "PVRPratyusha"


In [ ]:
!git config --global --list


user.email=pillalamarri.v@northeastern.edu
user.name=PVRPratyusha


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!git add .
!git commit -m "Rebuilt Milestone 2 branch with clean history (no secrets)"


/content/drive/MyDrive/generative-ai-room-decorator
[milestone2-colab 5a2784c] Rebuilt Milestone 2 branch with clean history (no secrets)
 8 files changed, 1659 deletions(-)
 delete mode 100644 .gitignore
 delete mode 100644 README_milestone2.md
 delete mode 100644 configs/.gitkeep
 delete mode 100644 data/.gitkeep
 delete mode 100644 notebooks/Generative_Project_Milestone_2.ipynb
 delete mode 100644 notebooks/generative_project_milestone_1.py
 delete mode 100644 outputs/.gitkeep
 delete mode 100644 src/.gitkeep


In [ ]:
!git push origin --delete milestone2-colab


error: unable to delete 'milestone2-colab': remote ref does not exist
error: failed to push some refs to 'https://github.com/PVRPratyusha/generative-ai-room-decorator.git'


In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!git push origin milestone2-colab --force


/content/drive/MyDrive/generative-ai-room-decorator
Enumerating objects: 18, done.
Counting objects: 100% (18/18), done.
Delta compression using up to 2 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (18/18), 585.98 KiB | 4.51 MiB/s, done.
Total 18 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), done.
remote: error: GH013: Repository rule violations found for refs/heads/milestone2-colab.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token 

In [ ]:
%cd /content/drive/MyDrive/generative-ai-room-decorator
!ls -R


/content/drive/MyDrive/generative-ai-room-decorator
.:


In [ ]:
!find "/content/drive/MyDrive" -maxdepth 4 -type d -name "generative-ai-room-decorator*"


/content/drive/MyDrive/generative-ai-room-decorator


In [ ]:
!find "/content/drive/MyDrive" -name "generative_project_milestone_1.py"


In [ ]:
!find "/content/drive/MyDrive" -name "Generative_Project_Milestone_2.ipynb"


In [ ]:
!find "/content/drive/MyDrive" -name ".gitkeep"


In [ ]:
!find "/content/drive/MyDrive" -maxdepth 4 -type d -name "configs"
!find "/content/drive/MyDrive" -maxdepth 4 -type d -name "notebooks"
!find "/content/drive/MyDrive" -maxdepth 4 -type d -name "src"
!find "/content/drive/MyDrive" -maxdepth 4 -type d -name "data"
!find "/content/drive/MyDrive" -maxdepth 4 -type d -name "outputs"


In [ ]:
!find "/content/drive/MyDrive" -maxdepth 5 -type f -name "*.py"
!find "/content/drive/MyDrive" -maxdepth 5 -type f -name "*.ipynb"
!find "/content/drive/MyDrive" -maxdepth 5 -type f -name "Generative*Milestone*"
!find "/content/drive/MyDrive" -maxdepth 5 -type f -name "*.gitkeep"


/content/drive/MyDrive/Colab Notebooks/Copy of Lab1_Features.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of Module1_Lab1_0473.ipynb
/content/drive/MyDrive/Colab Notebooks/Module1_Lab1_fmml20210473.ipynb
/content/drive/MyDrive/Colab Notebooks/Mod2_Lab1_fmml20210468.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of FMML_CLF1_Lab1.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of FMML_CLF1_Lab2.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of Project_FMML_CLF1.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of Build_a_food_image_classifier.ipynb
/content/drive/MyDrive/Colab Notebooks/Heart Attack -updated.ipynb
/content/drive/MyDrive/Colab Notebooks/Heart_Attack_analysis.ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled2.ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled3.ipynb
/content/drive/MyDrive/Colab Notebooks/Copy of NewsSentiment.ipynb
/content/drive/MyDrive/Colab Notebooks/NewsSentiment.ipynb
/content/drive/MyDrive/Colab Notebooks/FMML20210473_Mod4_Lab2 (

In [ ]:
!find "/content/drive/MyDrive" -maxdepth 5 -type f -name "*Milestone_2*.ipynb"


In [ ]:
!find "/content/drive/MyDrive" -maxdepth 5 -type d -name "configs"
!find "/content/drive/MyDrive" -maxdepth 5 -type d -name "src"
!find "/content/drive/MyDrive" -maxdepth 5 -type d -name "notebooks"
!find "/content/drive/MyDrive" -maxdepth 5 -type d -name "data"
!find "/content/drive/MyDrive" -maxdepth 5 -type d -name "outputs"


In [1]:
!find "/content/drive/MyDrive" -maxdepth 5 -type f -name "Generative_Project_Milestone2.ipynb"


find: ‘/content/drive/MyDrive’: No such file or directory
